In [1]:
import sys
sys.path.append('../')

In [2]:
from utils.data import MNIST_dataset
from utils.misc import (
    show_random_images,
    get_device,
    set_seed,
    is_cuda,
    show_model_summary,
)
from utils.model import Net
from utils.training import train
from utils.testing import test

In [7]:
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.optim as optim
import numpy as np

### Device

In [3]:
set_seed()
cuda = is_cuda()
device = get_device()

[INFO] seed set 1
[INFO] Cuda Avaliable :  False
[INFO] device :  cpu


## Data Loader

In [5]:
# train_loader = MNIST_dataset(
#     train=True, cuda=cuda
# ).get_loader()
# test_loader = MNIST_dataset(
#     train=False, cuda=cuda
# ).get_loader()

In [6]:
# show_random_images(train_loader)

## Load Model

In [8]:
net = Net(dropout_value=0.05, BN=True).to(device)
show_model_summary(net)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 26, 26]              54
              ReLU-2            [-1, 6, 26, 26]               0
           Dropout-3            [-1, 6, 26, 26]               0
       BatchNorm2d-4            [-1, 6, 26, 26]              12
            Conv2d-5           [-1, 12, 24, 24]             648
              ReLU-6           [-1, 12, 24, 24]               0
           Dropout-7           [-1, 12, 24, 24]               0
       BatchNorm2d-8           [-1, 12, 24, 24]              24
            Conv2d-9            [-1, 7, 24, 24]              84
        MaxPool2d-10            [-1, 7, 12, 12]               0
           Conv2d-11           [-1, 16, 10, 10]           1,008
             ReLU-12           [-1, 16, 10, 10]               0
          Dropout-13           [-1, 16, 10, 10]               0
      BatchNorm2d-14           [-1, 16,

## Training

In [9]:
L1 = False
L2 = False
weight_decay = 1e-5 if L2 else 0

optimizer = optim.SGD(
    net.parameters(), 
    lr=0.020, momentum=0.9, 
    weight_decay=weight_decay
)
lr_scheduler = ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, 
    patience=2, min_lr=0.001, verbose=True
)

In [ ]:
## training loop 
train_loss_batch, train_acc_batch = train(
    net, device, 
    train_loader, 
    optimizer, 
    lr_scheduler,
    L1=L1
)
train_acc = np.mean(train_acc_batch)
train_loss = np.mean(train_loss_batch)

test_loss, test_acc = test(
    net, device, 
    test_loader,
    lr_scheduler
)